# asyncio
- python3.4开始引入标准库当中,内置对异步io支持
- asyncio本身是一个消息循环
- 步骤:
    - 创建消息循环
    - 把协程导入
    -关闭

In [1]:
import threading
#引入异步io包
import asyncio

#使用协程
@asyncio.coroutine
def hello():
    print('Hello world!(%s)'%threading.currentThread())
    print('Start....(%s)'%threading.currentThread())
    yield from asyncio.sleep(5)
    print('Done....(%s)'%threading.currentThread())
    print('Hello again! (%s)'%threading.currentThread())
 
# 启动消息循环
loop=asyncio.get_event_loop()
#定义任务
tasks=[hello(),hello()]
#asyncio使用wait等待task
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world!(<_MainThread(MainThread, started 139859808741120)>)
Start....(<_MainThread(MainThread, started 139859808741120)>)
Hello world!(<_MainThread(MainThread, started 139859808741120)>)
Start....(<_MainThread(MainThread, started 139859808741120)>)
Done....(<_MainThread(MainThread, started 139859808741120)>)
Hello again! (<_MainThread(MainThread, started 139859808741120)>)
Done....(<_MainThread(MainThread, started 139859808741120)>)
Hello again! (<_MainThread(MainThread, started 139859808741120)>)


In [1]:
import asyncio
@asyncio.coroutine
def wget(host):
    print('wget %s...'%host)
    #异步请求网络地址
    connect=asyncio.open_connection(host,80)
    # 注意yield from的用法
    reader,writer=yield from connect
    header='GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line= yield from reader.readline()
        #http协议的换行使用\r\n
        if line ==b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8'.rstrip())))
    writer.close()
    
loop=asyncio.get_event_loop()
tasks=[wget(host) for host in ['www.sina.com.cn','www.souhu.com','www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

wget www.163.com...
wget www.sina.com.cn...
wget www.souhu.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily

www.sina.com.cn header > Server: nginx

www.sina.com.cn header > Date: Sat, 13 Oct 2018 03:50:07 GMT

www.sina.com.cn header > Content-Type: text/html

www.sina.com.cn header > Content-Length: 154

www.sina.com.cn header > Connection: close

www.sina.com.cn header > Location: https://www.sina.com.cn/

www.sina.com.cn header > X-Via-CDN: f=edge,s=cmcc.fujian.ha2ts4.183.nb.sinaedge.com,c=111.145.72.29;

www.sina.com.cn header > X-Via-Edge: 15394026071781d48916f9ffc057010e250f0

www.163.com header > HTTP/1.0 302 Moved Temporarily

www.163.com header > Server: Cdn Cache Server V2.0

www.163.com header > Date: Sat, 13 Oct 2018 03:50:07 GMT

www.163.com header > Content-Length: 0

www.163.com header > Location: http://www.163.com/special/0077jt/error_isp.html

www.163.com header > Connection: close



# asyncio and await
- 为了更好的表示异步io
- Python3.5引入
- 让协程代码更加简洁
- 使用上,可以单间的进行替换
    - 用async替换@asyncio.coroutine
    - await 替换yield from

# aiohttp
- asyncio实现单线程的并发io,在客户端用处不大
- 在服务器端可以asyncio+coroutine配合,因为http是io操作
- asyncio实现了tcp,udp,ssl等协议
- aiohttp是给予asyncio实现的http框架
- pip install aiohttp

In [4]:
#aiohttp案例
import asyncio
from aiohttp import web


async def index(request):
    await asyncio.sleep(0.5)
    return web.Response(body=b'<h1>Index</h1>')

async def hello(request):
    await asyncio.sleep(0.5)
    text='<h1>hello,%s!</h1>' % request.match_info['name']
    return web.Response(body=text.encode('utf-8'))

async def init(loop):
    app=web.Application(loop=loop)
    app.router.add_route('GET','/',index)
    app.router.add_route('GET','/hello/{name}',hello)
    srv=await loop.create_server(app.match_handler(),'127.0.0.1',8000)
    print('Serve  started at http:.//127.0.0.1:8000')
    return srv

loop=asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever

ModuleNotFoundError: No module named 'aiohttp'

# concurrent.futures
- python3新增的库
- 类似其他语言的线程池,概念
- 利用multiprocessiong实现真正的并行计算
- 核心原来:以子进程的形式,并行运行多个python解释器
    从而令python程序可以利用多核CPU来提升执行速度
    由于子进程与主解释器相分离,所以他们的全局解释器锁也是相互独立的
    每个子进程能够完整的使用一个CPU内核
- concurrent.futures.Executor
- ThreadPoolExecutor
- ProcessPoolExecutor
- 执行的时候需要自行选择
- submit(fn,args,kwargs)
    - fn:异步执行的函数
    - args,kwargs参数

In [3]:
#关于concurrent的案例
from concurrent.futures import ThreadPoolExecutor
import time

def return_future(msg):
    time.sleep(3)
    return msg

#创建一个线程池
pool=ThreadPoolExecutor(max_workers=2)

#往线程池加入2个task
f1=pool.submit(return_future,'hello')
f2=pool.submit(return_future,'world')

# 等待执行完毕
print(f1.done())
time.sleep(3)
print(f2.done())

# 结果
print(f1.result)
print(f2.result)

False
False
<bound method Future.result of <Future at 0x7f0bf953add8 state=running>>
<bound method Future.result of <Future at 0x7f0bf953aeb8 state=running>>


# current 中map函数
- map(fn,\*iterables,timeout=None)
    - 跟map函数类似
    - 函数需要进行异步执行
    - timeout:超时时间
    - map跟submit使用一个就行
    
# Future
    - 

In [6]:
# map案例
import time,re
import os,datetime
from concurrent import futures

data=['1','2']

def wait_on(argument):
    print(argument)
    time.sleep(2)
    return 'ok'

ex=futures.ThreadPoolExecutor(max_workers=2)
for i in ex.map(wait_on,data):
    print(i)

1
2
ok
ok
